In [4]:
import psycopg2
import os
import csv
import time
import numpy as np
import pandas as pd

from category_encoders import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GRU, Dropout, Input, BatchNormalization, Activation
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [5]:
conn = psycopg2.connect(
  host='localhost',
  dbname='train_db',
  user='project4',
  password='abcdqwer',
  port='5432'
)

In [6]:
dataset = pd.read_sql("SELECT * FROM train_cong", conn)
dataset_cp = dataset.copy()

conn.close()

/var/folders/5y/8vxkxcq10jbbp_61mxm4tcqw0000gn/T/ipykernel_90978/557139081.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataset = pd.read_sql("SELECT * FROM train_cong", conn)


In [7]:
dataset

,id,number_asc,week,line,st_code,st_name,clss,time,congestion
0,1,1,평일,1,150,서울역,상선,1000,26.6
1,2,1,평일,1,150,서울역,상선,1030,23.4
2,3,1,평일,1,150,서울역,상선,1100,22.4
3,4,1,평일,1,150,서울역,상선,1130,22.1
4,5,1,평일,1,150,서울역,상선,1200,21.5
...,...,...,...,...,...,...,...,...,...
63043,63044,1,일요일,8,2827,모란,하선,730,0.0
63044,63045,1,일요일,8,2827,모란,하선,800,0.0
63045,63046,1,일요일,8,2827,모란,하선,830,0.0
63046,63047,1,일요일,8,2827,모란,하선,900,0.0


In [8]:
dataset.drop(['id', 'number_asc', 'st_name', 'line'], axis=1, inplace=True, errors='ignore')
dataset.sort_values(['week', 'st_code', 'clss', 'time'], inplace=True)
dataset.reset_index(drop=True, inplace=True)

In [9]:
# dataset_cp[dataset_cp['line'] == 15][['st_code', 'st_name','clss', 'week']].value_counts()

In [10]:
# 이동 방향에 따른 역번호 증가
# 1호선(상선 증가) , 2호선(내선 증가), 3호선(하선 증가), 4호선(상선 증가), 5호선(하선 증가), 6호선(하선 증가), 7호선(하선 증가), 8(하선 증가)
# line 13(2호선), line 14(2호선), line 15(5호선)

dataset = dataset.replace({'평일':1, '토요일':2, '일요일':3, '상선':1, '하선':2, '내선':3, '외선':4})

In [11]:
dataset

,week,st_code,clss,time,congestion
0,3,150,1,530,2.4
1,3,150,1,600,6.9
2,3,150,1,630,6.8
3,3,150,1,700,6.1
4,3,150,1,730,7.5
...,...,...,...,...,...
63043,1,2827,2,2130,0.0
63044,1,2827,2,2200,0.0
63045,1,2827,2,2230,0.0
63046,1,2827,2,2300,0.0


In [12]:
x_train = dataset.drop('congestion', axis=1)
y_train = dataset['congestion']

In [13]:
x_train.shape, y_train.shape

((63048, 4), (63048,))

In [14]:
def base_model():
  inputs = Input(shape=(x_train.shape[1]))
  
  layer = Dense(256, kernel_initializer='he_normal')(inputs)
  layer = BatchNormalization()(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.2)(layer)
  
  layer = Dense(128, kernel_initializer='he_normal')(inputs)
  layer = BatchNormalization()(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.2)(layer)
  
  layer = Dense(64, kernel_initializer='he_normal')(inputs)
  layer = BatchNormalization()(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.2)(layer)
  
  outputs = Dense(1, name='output')(layer)
  
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss='mean_squared_error')
  
  return model

In [15]:
model = base_model()

model.fit(x_train, y_train, batch_size=128, epochs=30)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Epoch 1/30


2023-04-08 18:46:02.975477: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-08 18:46:02.975579: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-04-08 18:46:03.133121: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-08 18:46:03.289932: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


493/493 [==============================] - 3s 6ms/step - loss: 422.5395
Epoch 2/30
493/493 [==============================] - 3s 6ms/step - loss: 227.0821
Epoch 3/30
493/493 [==============================] - 3s 6ms/step - loss: 219.0296
Epoch 4/30
493/493 [==============================] - 3s 6ms/step - loss: 212.5102
Epoch 5/30
493/493 [==============================] - 3s 6ms/step - loss: 206.7061
Epoch 6/30
493/493 [==============================] - 3s 6ms/step - loss: 203.2556
Epoch 7/30
493/493 [==============================] - 3s 6ms/step - loss: 201.2077
Epoch 8/30
493/493 [==============================] - 3s 6ms/step - loss: 199.2867
Epoch 9/30
493/493 [==============================] - 3s 6ms/step - loss: 198.2651
Epoch 10/30
493/493 [==============================] - 3s 6ms/step - loss: 197.1409
Epoch 11/30
493/493 [==============================] - 3s 6ms/step - loss: 196.2353
Epoch 12/30
493/493 [==============================] - 3s 6ms/step - loss: 195.4487
Epoch 13/30


In [20]:
model.save('../ai_model/latest_model.h5')

In [21]:
model2 = tf.keras.models.load_model('../ai_model/latest_model.h5')

In [454]:
dataset_cp[(dataset_cp['st_code'] == 150) & (dataset_cp['time'] == 1800)]

,id,number_asc,week,line,st_code,st_name,clss,time,congestion
16,17,1,평일,1,150,서울역,상선,1800,30.8
53,54,1,평일,1,150,서울역,하선,1800,84.0
21032,21033,1,토요일,1,150,서울역,상선,1800,21.3
21069,21070,1,토요일,1,150,서울역,하선,1800,37.6
42048,42049,1,일요일,1,150,서울역,상선,1800,15.1
42085,42086,1,일요일,1,150,서울역,하선,1800,30.0


In [19]:
new_data = np.array([[1, 150, 2, 1800]])

model.predict(new_data)

1/1 [==============================] - 0s 10ms/step


array([[32.71419]], dtype=float32)

In [23]:
model2.predict(new_data)

1/1 [==============================] - 0s 9ms/step


array([[32.71419]], dtype=float32)

In [394]:
# seq_len = 37
# stride = 1
# batch_size = 37

# generator = TimeseriesGenerator(x_train, y_train, length=seq_len, stride=stride, batch_size=batch_size)

In [395]:
# def gru_model(layers=0):
#   inputs = Input(shape=(seq_len, x_train.shape[1]))
#   layer = GRU(64, activation='tanh')(inputs)
#   layer = Dropout(0.2)(layer)
  
#   for _ in range(layers):
#     layer = GRU(64, activation='tanh')(inputs)
#     layer = Dropout(0.2)(layer)
  
#   outputs = Dense(1, name='output')(layer)
  
#   model = Model(inputs=inputs, outputs=outputs)
  
#   model.compile(optimizer='adam', loss='mean_squared_error')
  
#   return model

In [ ]:
# model = gru_model(6)

# model.fit(generator, epochs=10)

In [ ]:
# new_data = np.array([[1, 1, 150, 1, 2000]])

# last_sequence = x_train[-(seq_len - 1):]
# new_sequence = np.concatenate((last_sequence, new_data), axis=0)

In [ ]:
# model.predict(new_data.reshape(1, 1, x_train.shape[1]))